#  كيفية التعامل مع ملفات البيانات الضخمة

الطرق الثلاث هي:

1.  **Pandas مع `chunksize`**:
2.  **Dask**: .
3.  ** compressed File Pandas مع `chunksize

## 0. الإعداد


In [21]:
!pip install kaggle
from google.colab import files
import dask.dataframe as dd
import pandas as pd
import time
import psutil
import os
import sys
import gzip
import shutil

uploaded = files.upload()
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
for fn in uploaded.keys():
    shutil.move(fn, os.path.expanduser("~/.kaggle/kaggle.json"))

!chmod 600 ~/.kaggle/kaggle.json
!kaggle config view
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

competition_name = 'talkingdata-adtracking-fraud-detection'
train_file = os.path.join(data_dir, 'train.csv')
compressed_file = os.path.join(data_dir, 'train.csv.gz')
zip_path = os.path.join(data_dir, f"{competition_name}.zip")
!kaggle competitions download -c {competition_name} -p {data_dir}
!unzip -j -o {zip_path} train.csv -d {data_dir} || unzip -j -o {zip_path} train_sample.csv -d {data_dir}




Saving kaggle.json to kaggle.json
Configuration values from /root/.kaggle
- username: bachirmeftah
- path: None
- proxy: None
- competition: None
talkingdata-adtracking-fraud-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./data/talkingdata-adtracking-fraud-detection.zip
  inflating: ./data/train.csv        


إنشاء ملف مضغوط من ملف البيانات الضخم.

In [22]:
target = train_file if os.path.exists(train_file) else os.path.join(data_dir, "train_sample.csv")

if os.path.exists(target):
    print(f"\n🗜️  Creating compressed file for: {target}")
    print("...this may take 10+ minutes, please wait...")

    with open(target, 'rb') as f_in:
        with gzip.open(compressed_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    print("✅ Setup complete! Compressed file saved at:", compressed_file)
else:
    print("❌ train.csv not found in the zip. Please check dataset contents.")


🗜️  Creating compressed file for: ./data/train.csv
...this may take 10+ minutes, please wait...
✅ Setup complete! Compressed file saved at: ./data/train.csv.gz


## 1 الطريقة الأولى:  Pandas مع `chunksize`
*italicized text*

In [26]:
process = psutil.Process(os.getpid())
chunk_size = 150_000
total_rows = 0


# بدء التوقيت
start_time = time.time()
start_mem = process.memory_info().rss / (1024 * 1024)
peak_mem = start_mem # تهيئة ذروة الذاكرة
print(f"Starting RAM: {start_mem:.2f} MB")

try:
    chunk_iterator = pd.read_csv(train_file, chunksize=chunk_size)

    # التكرار عبر كل قطعة
    for chunk in chunk_iterator:
        total_rows += len(chunk)

        current_mem = process.memory_info().rss / (1024 * 1024)
        if current_mem > peak_mem:
            peak_mem = current_mem

    # إيقاف التوقيت
    end_time = time.time()
    time_taken = end_time - start_time
    end_mem = process.memory_info().rss / (1024 * 1024)

    print("\n--- Results for Method 1 ---")
    print(f"Total rows: {total_rows}")
    print(f"Time taken: {time_taken:.2f} seconds")
    print(f"Peak RAM usage: {peak_mem:.2f} MB")
    print(f"Ending RAM usage: {end_mem:.2f} MB")

except FileNotFoundError:
    print(f"ERROR: File not found at {train_file}. Please check the download step.")

Starting RAM: 415.20 MB


/tmp/ipython-input-4140004620.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iterator:



--- Results for Method 1 ---
Total rows: 184903890
Time taken: 140.84 seconds
Peak RAM usage: 415.20 MB
Ending RAM usage: 415.20 MB


##2. الطريقة الثانية:  Dask


In [27]:
process = psutil.Process(os.getpid())


# بدء التوقيت
start_time = time.time()
start_mem = process.memory_info().rss / (1024 * 1024) # تحويل إلى ميجابايت
print(f"Starting RAM: {start_mem:.2f} MB")

try:
    ddf = dd.read_csv(
        train_file,
        dtype={'attributed_time': 'object'},
        blocksize="8MB"
    )

    ram_before_compute = process.memory_info().rss / (1024 * 1024)
    print(f"RAM after Dask setup (before compute): {ram_before_compute:.2f} MB")

    # هذا الأمر يشغل الحساب الفعلي
    total_rows = len(ddf)

    # إيقاف التوقيت
    end_time = time.time()
    time_taken = end_time - start_time
    ram_after_compute = process.memory_info().rss / (1024 * 1024)

    print("\n--- Results for Method 2 ---")
    print(f"Total rows: {total_rows}")
    print(f"Time taken: {time_taken:.2f} seconds")
    print(f"Peak RAM usage (after compute): {ram_after_compute:.2f} MB")

except FileNotFoundError:
    print(f"ERROR: File not found at {train_file}. Please check the download step.")

Starting RAM: 402.32 MB
RAM after Dask setup (before compute): 402.32 MB

--- Results for Method 2 ---
Total rows: 184903890
Time taken: 161.28 seconds
Peak RAM usage (after compute): 422.33 MB


## 3 الطريقة الثالثة: compressed File Pandas مع `chunksize`

In [28]:
process = psutil.Process(os.getpid())
chunk_size = 150_000
total_rows = 0


# بدء التوقيت
start_time = time.time()
start_mem = process.memory_info().rss / (1024 * 1024)
peak_mem = start_mem # تهيئة
print(f"Starting RAM: {start_mem:.2f} MB")

try:
    with pd.read_csv(
        compressed_file,
        compression='gzip',
        dtype={'attributed_time': 'object'},
        chunksize=chunk_size,
        on_bad_lines='skip'
    ) as reader:

        for chunk in reader:
            total_rows += len(chunk)

            current_mem = process.memory_info().rss / (1024 * 1024)
            if current_mem > peak_mem:
                peak_mem = current_mem


    # إيقاف التوقيت
    end_time = time.time()
    time_taken = end_time - start_time
    end_mem = process.memory_info().rss / (1024 * 1024)

    print("\n--- Results for Method 3 ---")
    print(f"Total rows: {total_rows}")
    print(f"Time taken: {time_taken:.2f} seconds")
    print(f"Peak RAM usage: {peak_mem:.2f} MB")
    print(f"Ending RAM usage: {end_mem:.2f} MB")

except FileNotFoundError:
    print(f"ERROR: File not found at {compressed_file}. Please check the compression step.")

Starting RAM: 422.33 MB

--- Results for Method 3 ---
Total rows: 184903890
Time taken: 193.72 seconds
Peak RAM usage: 436.70 MB
Ending RAM usage: 436.70 MB
